# Decoupling Coupled Harmonic Oscillators

by: Tianming Zhao, Ximeng Song

## Introduction


Harmonic Oscillators are one of the foundations of Physics as a whole. Many physical problems can be traced back to this basic solution. Though some are easier to solve, such as the mathematical pendulum, and others are much more complex. Most notably, Harmonic Oscillators may be coupled, which means that the parameters of one harmonic oscialltor influences the equations of motion of other harmonic oscillators. Though that does mean there are more equations and variables to solve, but as most systems of equations, these ones can also be simplified through the use of matrices.

This is where SageMath comes in. We will be using SageMath as a tool to decouple Harmonic Oscillators by finding generalized coordinates. Since only basic matrix operations will be needed, using SageMath is a effective way to shorten the calculation processes required. Though, as we will see, certain complications do arise when using SageMath's usual built in functions.

## The Physics

An ideal system in motion usually has two types of energy: Kinetic energy and potential energy. The most important fact is that the total energy remains constant. But because we are observing coupled harmonic oscillators, there will be multiple kinetic energies and multiple potential energies to consider. These will influence each other in more complex ways than the classic harmonic oscillator, since only the overall energy of all energies added together reamins constant.

These influences between the masses can be represented by a matrix, as we will show in the following.

## Formulas:

#### Kinetic Energy

$T = \frac{1}{2} mv^2$

#### Potential Energy

$V = \frac{1}{2} kx^2$


#### General matrix form:

$T = \frac{1}{2}\dot{\vec{q}}^\intercal\mathcal{T}\dot{\vec{q}} $

$V = \frac{1}{2}\vec{q}^\intercal\mathcal{V}\vec{q} $

## Example Problem:

#### Kinetic Energy:
$T = \frac{1}{2}m(\dot{q_1}^2+\dot{q_2}^2)$

#### Potential Energy
$V = \frac{1}{2}k(q_1^2+q_2^2)+\frac{1}{2}k_{12}(q_2-q_1)^2$

#### General matrix form:
$T = \frac{1}{2}\dot{\vec{q}}^\intercal\begin{pmatrix}
        m & 0 \\
        0 & m \\
    \end{pmatrix}\dot{\vec{q}}$
    
$V = \frac{1}{2}\vec{q}^\intercal\begin{pmatrix}
        k+k_{12} & -k_{12} \\
        -k_{12} & k+k_{12} \\
    \end{pmatrix}\vec{q}$

In [26]:
# input
var('m', 'k', 'k12', 't')
di = 2
T = matrix(SR, di, di, [m, 0,
                        0, m])
show(T)
V = matrix(SR, di, di, [k + k12, -k12,
                        -k12, k + k12])
show(V)

[m 0]
[0 m]

[k + k12    -k12]
[   -k12 k + k12]

In [27]:
# proof of concept
var('dq1', 'dq2', 'q1', 'q2')
dq = column_matrix(SR, [dq1, dq2])
q = column_matrix(SR, [q1, q2])
show(1/2*dq.transpose()*T*dq)
show((1/2*q.transpose()*V*q).simplify_full())

[1/2*dq1^2*m + 1/2*dq2^2*m]

[1/2*(k + k12)*q1^2 - k12*q1*q2 + 1/2*(k + k12)*q2^2]

To solve the decouple the Coupled Harmonic Oscillator, we need the eigenvalues of $\mathcal{V}$ in respect to $\mathcal{T}$. This proves to be more of a challenge than usual, since SageMath built in functions such as eigenvalues() do not support eigenvalues in respect matrices other than the identity matrix.

This means we need to go through the steps of calculating eigenvalues from scratch, only using basic functions while doing so.

In [28]:
M = V-t*T
P = M.determinant()
show(P)
l = P.roots(t)
show(l)

(m*t - k - k12)^2 - k12^2

[((k + 2*k12)/m, 1), (k/m, 1)]

This solves part of the equations already, since the eigenvalues are the squared resonance frequencies.
The new equations are now

$\ddot{\xi_k}+\lambda_k\xi _k = 0$,

where $\lambda_k$ are the eigenvalues that we have just calculated, and $\xi_k$ are the components of the generalized coordinate $\vec{\xi}$.

Most importantly, $\xi_k$ are decoupled from each other which means that these equations are homogenous differential equations of the second order with known solutions, namely  exponential functions. 

Now the only missing equation to finally find out the decoupled equations of movement is the relationship between $\vec{q}$ and $\vec{\xi}$. The first step to archive this is to calculate the eigenvectors of $\mathcal{V}$ in respect to $\mathcal{T}$.

In [29]:
base = []
for val in l:
    R = V-val[0]*T
    for x in R.right_kernel().basis():
        base.append(matrix(x.simplify_full()).transpose())
show(base)

[
[ 1]  [1]
[-1], [1]
]

Furthermore, the eigenvectors need to be normalized in respect to $\mathcal{T}$ as well, which SageMath does not support either. Once again, we need to use the basic mathematical concept to do so.

In [30]:
norm_eig_vec = []
for b in base:
    norm = b.transpose()*T*b
    norm_eig_vec.append((norm[0,0]**(-1/2)*b).simplify_full())
show(norm_eig_vec)

[
[ 1/2*sqrt(2)/sqrt(m)]  [1/2*sqrt(2)/sqrt(m)]
[-1/2*sqrt(2)/sqrt(m)], [1/2*sqrt(2)/sqrt(m)]
]

These normalized eigenvectors form the base transformation matrix that is orthonormal in respect to $\mathcal{T}$, and diagonalizes $\mathcal{V}$. Which means that it does not change the physical quantities. We will call this matrix $\overline{\mathcal{B}}$.

In [31]:
B_ = matrix(SR, di, di, [i[0] for col in norm_eig_vec for i in col]).transpose()
show(B_)

[ 1/2*sqrt(2)/sqrt(m)  1/2*sqrt(2)/sqrt(m)]
[-1/2*sqrt(2)/sqrt(m)  1/2*sqrt(2)/sqrt(m)]

To proof that this is indeed the case, we show that $\overline{\mathcal{B}}^\intercal\mathcal{T}\overline{\mathcal{B}}$ is the identity matrix, and that $\overline{\mathcal{B}}^{\intercal^{-1}}\overline{\mathcal{B}}^{-1}$ is $\mathcal{T}$.

In [32]:
# proof of concept
show((B_.transpose()*T*B_).simplify_full())
show((B_.transpose().inverse()*B_.inverse()).simplify_full())

[1 0]
[0 1]

[m 0]
[0 m]

Since $\overline{\mathcal{B}}^\intercal\mathcal{T}\overline{\mathcal{B}}$ is the identity matrix and $\vec{\xi}$ is the generalized coordinate, we can now combine this with the general matrix from to obtain

$T = \frac{1}{2}\dot{\vec{q}}^\intercal\mathcal{T}\dot{\vec{q}} = \frac{1}{2}\dot{\vec{\xi}}^\intercal\dot{\vec{\xi}} = \frac{1}{2}\dot{\vec{\xi}}^\intercal\overline{\mathcal{B}}^\intercal\mathcal{T}\overline{\mathcal{B}}\dot{\vec{\xi}}$

and thus

$\vec{q} = \overline{\mathcal{B}}\vec{\xi}$.

We now show how $\overline{\mathcal{B}}$ describes the relation between $\vec{q}$ and $\vec{\xi}$ for this example.

In [33]:
# proof of concept
xi = B_.inverse()*q
show(xi)

[1/2*sqrt(2)*sqrt(m)*q1 - 1/2*sqrt(2)*sqrt(m)*q2]
[1/2*sqrt(2)*sqrt(m)*q1 + 1/2*sqrt(2)*sqrt(m)*q2]

In [34]:
# proof of concept
var('xi1', 'xi2')
xi = column_matrix(SR, [xi1, xi2])
show(B_*xi)

[ 1/2*sqrt(2)*xi1/sqrt(m) + 1/2*sqrt(2)*xi2/sqrt(m)]
[-1/2*sqrt(2)*xi1/sqrt(m) + 1/2*sqrt(2)*xi2/sqrt(m)]